In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Reshape, Flatten, Dropout, Input
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D , Conv2DTranspose
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os


In [ ]:
def load_data(image_dir, img_size=(64, 64)):
    datagen = ImageDataGenerator(rescale=1./255)
    data_flow = datagen.flow_from_directory(image_dir, target_size=img_size, batch_size=128, class_mode=None)
    return data_flow

# Path to the directory containing the CelebA dataset
image_dir = '/kaggle/input/celeba-dataset/img_align_celeba'
data_flow = load_data(image_dir)


In [ ]:
def def_generator():
    model = Sequential()
    model.add(Dense(256 * 8 * 8, activation="relu", input_dim=100))
    model.add(Reshape((8, 8, 256)))
    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2D(3, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))
    return model

def def_discriminator():
    model = Sequential()
    model.add(Conv2D(64, kernel_size=3, strides=2, input_shape=(64, 64, 3), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(512, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

def def_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = Input(shape=(100,))
    x = generator(gan_input)
    gan_output = discriminator(x)
    gan = Model(gan_input, gan_output)
    return gan


In [ ]:
def train_gan(generator, discriminator, gan, data_flow, epochs=10000, batch_size=128, save_interval=200):
    half_batch = batch_size // 2

    for epoch in range(epochs):
        # Train Discriminator
        real_images = next(data_flow)  # Correct way to get the next batch
        real_images = real_images[:half_batch]
        
        noise = np.random.normal(0, 1, (half_batch, 100))
        fake_images = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(real_images, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(fake_images, np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, 100))
        valid_y = np.array([1] * batch_size)
        g_loss = gan.train_on_batch(noise, valid_y)

        # Print progress
        print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100 * d_loss[1]}%] [G loss: {g_loss}]")

        # Save generated images at intervals
        if epoch % save_interval == 0:
            save_images(generator, epoch)

def save_images(generator, epoch, save_dir='/kaggle/working/gan_images'):
    os.makedirs(save_dir, exist_ok=True)
    noise = np.random.normal(0, 1, (25, 100))
    gen_images = generator.predict(noise)
    gen_images = 0.5 * gen_images + 0.5  # Rescale to 0-1

    fig, axs = plt.subplots(5, 5)
    count = 0
    for i in range(5):
        for j in range(5):
            axs[i, j].imshow(gen_images[count])
            axs[i, j].axis('off')
            count += 1
    fig.savefig(os.path.join(save_dir, f"epoch_{epoch}.png"))
    plt.close()

# Build and compile the discriminator
discriminator = def_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Build and compile the generator
generator = def_generator()

# Build and compile the GAN
gan = def_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam())

# Train the GAN ,,,, 10000 epochs recommended
train_gan(generator, discriminator, gan, data_flow, epochs=100, batch_size=128, save_interval=200)


In [ ]:
# Generate and display two fake faces
import numpy as np
noise = np.random.normal(0, 1, (2, 100))
gen_images = generator.predict(noise)
gen_images = 0.5 * gen_images + 0.5  # Rescale to 0-1

for i in range(2):
    plt.imshow(gen_images[i])
    plt.axis('off')
    plt.show()
